# Postprocess predictions and create a submission

In [1]:
#@title Submission notes and params { run: "auto" }
version_notes = "BBox reduction" #@param {type:"string"}

reduce_method = "weighted_boxes_fusion" #@param ["None", "nms", "soft_nms", "non_maximum_weighted", "weighted_boxes_fusion"]

iou_threshold = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
skip_box_thrreshold = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}
sigma = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

binary_abnormal_threshold = 0.05 #@param {type:"slider", min:0, max:1, step:0.01}
binary_normal_threshold = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}
assert(binary_abnormal_threshold <= binary_normal_threshold)

In [2]:
#@title Select models versions { run: "auto" }

binary_version = "6: EfficientNetB3, include probabilities (for later double thresholding); image size: (1024, 1024); threshold: 0.8" #@param ["2: Try 1024 resolution", "3: EffNet, 1024 resolution, threshold 0.8", "4: EffNet, 1024 resolution, threshold 0.8", "5: EffNet, 512 resolution, threshold 0.8", "6: EfficientNetB3, include probabilities (for later double thresholding); image size: (1024, 1024); threshold: 0.8"]
binary_notes = binary_version.split(":")[1]
binary_version = int(binary_version.split(":")[0])


yolo_version_5_selected = False #@param {type: "boolean"}
yolo_version_6_selected = False #@param {type: "boolean"}
yolo_version_7_selected = False #@param {type: "boolean"}
yolo_version_8_selected = False #@param {type: "boolean"}
yolo_version_9_selected = False #@param {type: "boolean"}
yolo_version_10_selected = False #@param {type: "boolean"}
yolo_version_11_selected = False #@param {type: "boolean"}
yolo_version_12_selected = True #@param {type: "boolean"}
yolo_version_13_selected = True #@param {type: "boolean"}
yolo_version_14_selected = True #@param {type: "boolean"}
yolo_version_15_selected = True #@param {type: "boolean"}
yolo_version_16_selected = True #@param {type: "boolean"}

In [3]:
INPUTS_YOLO_VERSIONS = [
    {
        "version": 5,
        "version_notes": "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        # "path": f"v{version}",
        # "enabled": locals()[f"yolo_version_{version}_enabled"],
    },
    {
        "version": 6,
        "version_notes": "WBF preproc, YOLOv5x, 50 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
    },
    {
        "version": 7,
        "version_notes": "WBF preproc, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1",
    },
    {
        "version": 8,
        "version_notes": "WBF preproc with IoU 0.4, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15, remove (most) augs",
    },
    {
        "version": 9,
        "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs",
    },
    {
        "version": 10,
        "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 25 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs",
    },
    {
        "version": 11,
        "version_notes": "Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05",
    },
    {
        "version": 12,
        "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1",
    },
    {
        "version": 13,
        "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1",
    },
    {
        "version": 14,
        "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1",
    },
    {
        "version": 15,
        "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1",
    },
    {
        "version": 16,
        "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 4/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1",
    },
]

# Check that versions are distinct (no need to ensemble the same model)
assert(len(set(yolo_version["version"] for yolo_version in INPUTS_YOLO_VERSIONS)) == len(INPUTS_YOLO_VERSIONS))

for yolo_version in INPUTS_YOLO_VERSIONS:
    # Check for errors in fields.
    assert(all(key_name in yolo_version for key_name in ["version", "version_notes"]))

    yolo_version["path"] = f"v{yolo_version['version']}"
    yolo_version["enabled"] = locals()[f"yolo_version_{yolo_version['version']}_selected"]

# Filter only enabled.
INPUTS_YOLO_VERSIONS = [yolo_version for yolo_version in INPUTS_YOLO_VERSIONS if yolo_version["enabled"]]

## Setup

In [4]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/working', '/kaggle/input']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [5]:
from pathlib import Path

from IPython.display import clear_output

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = WORK_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

for yolo_version in INPUTS_YOLO_VERSIONS:
    yolo_version["path"] = INPUT_FOLDER_YOLO_OUT / yolo_version["path"]

print(f"Using the following YOLO versions:")
print("\n".join(str(yolo_version) for yolo_version in INPUTS_YOLO_VERSIONS))

Using the following YOLO versions:
{'version': 12, 'version_notes': 'Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1', 'path': PosixPath('/kaggle/working/vbdyolo-out/v12'), 'enabled': True}
{'version': 13, 'version_notes': 'Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1', 'path': PosixPath('/kaggle/working/vbdyolo-out/v13'), 'enabled': True}
{'version': 14, 'version_notes': 'Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1', 'path': PosixPath('/kaggle/working/vbdyolo-out/v14'), 'enabled': True}
{'version': 15, 'version_notes': 'Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs

## Get data from Kaggle

In [6]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@fix-datasets-download-file-unzip
clear_output()

In [7]:
!kaggle datasets download corochann/vinbigdata-chest-xray-original-png -f test_meta.csv -p {INPUT_FOLDER_ORIGINAL_PNG} --unzip

  0% 0.00/126k [00:00<?, ?B/s]
100% 126k/126k [00:00<00:00, 53.7MB/s]


In [8]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

In [9]:
for yolo_version in INPUTS_YOLO_VERSIONS:
    !rm -rf {yolo_version["path"]}
    !kaggle datasets download "witalia/vbdyolo-out-newest" -v {yolo_version["version"]} -p {yolo_version["path"]} --unzip --force

 93% 289M/310M [00:01<00:00, 234MB/s]
100% 310M/310M [00:01<00:00, 200MB/s]
 85% 132M/155M [00:00<00:00, 209MB/s]
100% 155M/155M [00:00<00:00, 230MB/s]
 88% 137M/155M [00:01<00:00, 126MB/s]
100% 155M/155M [00:01<00:00, 136MB/s]
 94% 146M/155M [00:00<00:00, 223MB/s]
100% 155M/155M [00:00<00:00, 248MB/s]
 87% 135M/155M [00:00<00:00, 222MB/s]
100% 155M/155M [00:00<00:00, 236MB/s]


In [10]:
!rm -rf {INPUT_FOLDER_BINARY}
!kaggle datasets download "witalia/vbdbinary" -v {binary_version} -p {INPUT_FOLDER_BINARY} --unzip --force

 72% 38.0M/52.7M [00:00<00:00, 117MB/s] 
100% 52.7M/52.7M [00:00<00:00, 220MB/s]


## Process YOLO output

### BBox reduction (potentially from multiple models)

In [11]:
!pip install ensemble-boxes
clear_output()

In [12]:
from ensemble_boxes import *
from typing import List

def reduce_boxes(image_labels_list: List[pd.DataFrame], iou_thr=iou_threshold, skip_box_thr=skip_box_thrreshold, sigma=sigma, method=None):
    label_empty = [len(df) == 0 for df in image_labels_list]

    if len(image_labels_list) == 0 or all(label_empty):
        # If there's nothing to fuse, return empty.
        return pd.DataFrame(columns=["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Ensemble_boxes doesn't do well with empty inputs, so skip those.
    labels = [image_labels["class_id"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    scores = [image_labels["conf"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    boxes = [image_labels[["x_min", "y_min", "x_max", "y_max"]].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]

    if method == non_maximum_weighted or method == weighted_boxes_fusion:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    elif method == soft_nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma)
    elif method == nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr)
    else:
        if len(labels) > 1:
            raise ValueError("Cannot fuse more than 1 model without a fusing method")
        [boxes], [scores], [labels] = boxes, scores, labels

    reduced_labels = pd.DataFrame().reindex(list(range(len(labels))))
    reduced_labels["class_id"] = np.array(labels).astype(np.int32)
    reduced_labels["conf"] = scores
    reduced_labels[["x_min", "y_min", "x_max", "y_max"]] = boxes

    return reduced_labels

### Read and transform labels

In [13]:
def read_prediction_labels(filename: Path):
    yolo_columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]
    if filename.exists():
        labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
        labels.columns = yolo_columns
    else:
        labels: pd.DataFrame = pd.DataFrame(columns=yolo_columns)

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = (labels["x_centre"] - labels["bw"] / 2).clip(0, 1)
    labels["y_min"] = (labels["y_centre"] - labels["bh"] / 2).clip(0, 1)
    labels["x_max"] = (labels["x_centre"] + labels["bw"] / 2).clip(0, 1)
    labels["y_max"] = (labels["y_centre"] + labels["bh"] / 2).clip(0, 1)
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    return labels


def scale_labels(labels: pd.DataFrame, image_w: int, image_h: int):
    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    return labels


def labels_to_string(labels: pd.DataFrame):
    # Empty DataFrame means there was no YOLO output file or "reduce" removed all bboxes.
    # Means "No finding"
    if len(labels) == 0:
        return "14 1 0 0 1 1"
    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

### Process YOLO output files, and calculate the predictions

In [14]:
import ensemble_boxes

results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    # Read YOLO output to pandas DataFrame.
    prediction_labels = [read_prediction_labels(
        yolo_version["path"] / "labels_pred" / f"{image_id}.txt"
    ) for yolo_version in INPUTS_YOLO_VERSIONS]

    # Reduce bboxes.
    method = None if reduce_method == "None" else getattr(ensemble_boxes, reduce_method)
    prediction_labels = reduce_boxes(prediction_labels, method=method)

    # Scale them to the original size.
    # NOTE: dim0 and dim1 are reversed: y-axis, x-axis!
    prediction_labels = scale_labels(prediction_labels, image_dims["dim1"], image_dims["dim0"])

    # Convert resulting bboxes from DataFrame to string.
    prediction_str = labels_to_string(prediction_labels)

    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10, random_state=10)

,image_id,PredictionString
1779,9b41c13ac918adcc1261f750a1d61b40,11 0.460571 681 464 856 534 8 0.369238 421 176...
341,202dc97c4b3ec7e50f4813456938500a,0 0.833203 967 634 1172 844 3 0.821387 699 121...
1276,711dc542c6c7e2b21889838cd050a810,7 0.173767 1365 978 1756 1550 7 0.091064 440 1...
1012,5ba0a669fd3430e9386758e838210b98,0 0.305566 1795 1177 2074 1485 7 0.228565 924 ...
470,2b2810284313c5547241b38668f67b2c,0 0.823926 1081 964 1355 1278 8 0.576025 392 1...
1755,990b0dcd83353894c4a2e73738cd6c25,3 0.114075 999 1146 2150 1716 5 0.057312 1508 ...
2773,ec6c4e81b448275577d3e4a74acbe443,0 0.772786 1092 690 1319 940 3 0.616553 852 13...
1085,6100bc75e0a668c382d92d5eb9038076,3 0.711426 669 1109 1547 1393 0 0.231250 1022 ...
2055,b1a596318e7bfdcbb74044bf6d38c6ed,0 0.555664 960 588 1157 793 3 0.514062 640 125...
2585,de7aa2b6adfb9db39489c019f5f3a1a1,3 0.838281 682 1374 1790 1741 0 0.825879 1035 ...


## Merge with Binary classifier output

In [15]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")

if "confidence" in results_df.columns:
    # If confidence of "No finding" is very high, replace with "No finding"
    results_df.loc[results_df["confidence"] >= binary_normal_threshold, "PredictionString"] = "14 1 0 0 1 1"
    # If confidence of "No finding" is medium, add "No finding" with confidence to the object detector's bboxes.
    # Skip those that are already "No finding".
    results_df.loc[
            (results_df["confidence"] < binary_normal_threshold) & \
            (results_df["confidence"] > binary_abnormal_threshold) & \
            (~(results_df["PredictionString"] == "14 1 0 0 1 1")),
        "PredictionString"] += results_df["confidence"].apply(lambda prob: f" 14 {prob:.6f} 0 0 1 1")
    # If confidence of "No finding" is low, i.e. confidence of abnormality is high, do nothing.
else:
    # Just use binary classifier's suggestion as to which to set as "No finding"
    results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"

results_df = results_df[["image_id", "PredictionString"]]

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name,confidence
0,980141b50ada8a624c9a1c34c91c95d8,normal,0.851390
1,f362fd880b0149646ccbb760be5f6354,normal,0.955708
2,0291515f5d14c34180a15712a55bf7bd,abnormal,0.111645
3,6cf8916303a07f3909297be170f21f7a,normal,0.946124
4,c6829051ecb281656c987fa2cfe5c706,abnormal,0.234600


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.692149 1255 589 1532 900 14 0.339869 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,3 0.833008 1099 1403 1933 1793 0 0.810547 1428...
3,009bc039326338823ca3aa84381f17f1,3 0.777441 666 1054 1561 1348 0 0.724609 987 4...
4,00a2145de1886cb9eb88869c85d74080,3 0.818262 782 1291 1859 1647 0 0.546484 1109 ...


## Submission overview

Total number of "No finding" before binary classifier filter

In [16]:
print((results_yolo_df["PredictionString"] == "14 1 0 0 1 1").sum())

140


Total number of "No finding"

In [17]:
(results_df["PredictionString"] == "14 1 0 0 1 1").sum()

1137

Total with class "No finding" appended

In [18]:
print(results_df["PredictionString"].str.contains(r" 14 \d\.\d+ 0 0 1 1").sum())

1797


## Submit to Kaggle

In [19]:
submission_message = " ***** ".join([
    version_notes,
    "; ".join([
        f"Reduce method: {reduce_method}",
        f"IoU {iou_threshold}",
        f"Conf {skip_box_thrreshold}",
        f"Sigma {sigma}",
        f"Binary Normal Thresh: {binary_normal_threshold}",
        f"Binary Abnormal Thresh: {binary_abnormal_threshold}",
    ]),
    f"Binary CLF: {binary_version} -{binary_notes}",
    "Ensemble YOLO: " + "; ".join(f"{yolo_version['version']} - {yolo_version['version_notes']}" for yolo_version in INPUTS_YOLO_VERSIONS),
])
!echo "{submission_message}"
!kaggle competitions submit \
    vinbigdata-chest-xray-abnormalities-detection \
    -f {WORK_FOLDER}/submission.csv \
    -m "{submission_message}"
!sleep 10
!kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

BBox reduction ***** Reduce method: weighted_boxes_fusion; IoU 0.6; Conf 0.1; Sigma 0.1; Binary Normal Thresh: 0.95; Binary Abnormal Thresh: 0.05 ***** Binary CLF: 6 - EfficientNetB3, include probabilities (for later double thresholding); image size ***** Ensemble YOLO: 12 - Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1; 13 - Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1; 14 - Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1; 15 - Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1; 16 - Augs: remove